In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import requests 
from bs4 import BeautifulSoup 

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [2]:
CLIENT_ID = 'UXWV2V2VG3VGA3O10HEBJIKLMVERAKIVO45VDRHIA0XBNOJH' # your Foursquare ID
CLIENT_SECRET = 'STQCNT0UM1BMGR354CNXQU4P035Y3RU0AABKXK4TPNODES0R' # your Foursquare Secret
ACCESS_TOKEN = 'PBNLWQG4HXOXBIX2BPN0FB42X20O5TCPZET4WOKFGPHB1OGM' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UXWV2V2VG3VGA3O10HEBJIKLMVERAKIVO45VDRHIA0XBNOJH
CLIENT_SECRET:STQCNT0UM1BMGR354CNXQU4P035Y3RU0AABKXK4TPNODES0R


In [3]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(URL) 
  
soup = BeautifulSoup(page.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

#https://realpython.com/beautiful-soup-web-scraper-python/
#url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#wikipedia_page = urllib.requests.urlopen(url)
#df = pd.read_html(wikipedia_page.content, header=0) [0]
#df.head()
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

Page Scrapped.


In [4]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = pd.read_html(page.content, header=0) [0]
df.rename(index=str, columns={"Postal Code": "PostalCode"}, inplace = True)
df.rename(index=str, columns={"Neighbourhood": "Neighborhood"}, inplace= True)
df.head()
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

#df_2 = df[df.Borough != 'Not assigned'] 
#df_2.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:

#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df_2 = df[df.Borough != 'Not assigned'].reset_index(drop=True) 
df_2.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:

#More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: 
#Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

df_3 = df_2.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))
df_3.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
replace=df_3['Neighborhood']=="Not assigned"
df_3.loc[replace, 'Neighborhood']=df_3.loc[replace, 'Borough']
df_3.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
df_3.shape

(103, 3)

In [9]:

path= 'http://cocl.us/Geospatial_data'
data=pd.read_csv(path)
data.rename(index=str, columns={"Postal Code": "PostalCode"}, inplace = True)
data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
data2=pd.merge(df_3, data)

data2
#Combined both dataframes

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [11]:

CLIENT_ID = 'UXWV2V2VG3VGA3O10HEBJIKLMVERAKIVO45VDRHIA0XBNOJH' # your Foursquare ID
CLIENT_SECRET = 'STQCNT0UM1BMGR354CNXQU4P035Y3RU0AABKXK4TPNODES0R' # your Foursquare Secret
ACCESS_TOKEN = 'PBNLWQG4HXOXBIX2BPN0FB42X20O5TCPZET4WOKFGPHB1OGM' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UXWV2V2VG3VGA3O10HEBJIKLMVERAKIVO45VDRHIA0XBNOJH
CLIENT_SECRET:STQCNT0UM1BMGR354CNXQU4P035Y3RU0AABKXK4TPNODES0R


In [24]:
neighborhood_name = data2.loc[0, 'Neighborhood'] # neighborhood name
neighborhood_latitude = data2.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = data2.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

radius = 500 # define radius
LIMIT = 75 # limit of number of venues returned by Foursquare API


url = 'https://api.foursquare.com/v2/venues/explore?&client_id=UXWV2V2VG3VGA3O10HEBJIKLMVERAKIVO45VDRHIA0XBNOJH&client_secret=STQCNT0UM1BMGR354CNXQU4P035Y3RU0AABKXK4TPNODES0R&v=20180605&ll=43.806686,-79.194353&radius=500&limit=75'

results = requests.get(url).json()
results

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


{'meta': {'code': 200, 'requestId': '60007cf51cd7b4099973d9c1'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111860045, 'lng': -79.18812918076166},
   'sw': {'lat': 43.8021859955, 'lng': -79.20057681923835}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display',
 

In [26]:
Etobicoke_data = data2[data2['Borough'] == 'Etobicoke'].reset_index(drop=True)
Etobicoke_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.605647,-79.501321
1,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484
2,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
3,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
4,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999


In [28]:
address = 'Etobicoke, ON'

geolocator = Nominatim(user_agent="ON_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(latitude, longitude))



The geograpical coordinate of Etobicoke are 43.6435559, -79.5656326.


In [36]:
map_Etobicoke= folium.Map(location=[latitude, longitude], zoom_start= 11)
# add markers to map
for lat, lng, label in zip(Etobicoke_data['Latitude'], Etobicoke_data['Longitude'], Etobicoke_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Etobicoke)  
    
map_Etobicoke


In [31]:


Scarborough_data = data2[data2['Borough'] == 'Scarborough'].reset_index(drop=True)
Scarborough_data.head()



map_Scarborough= folium.Map(location=[latitude, longitude], zoom_start= 8)
# add markers to map
for lat, lng, label in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'], Scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough


In [35]:
map_data2= folium.Map(location=[latitude, longitude], zoom_start= 12)
# add markers to map
for lat, lng, label in zip(data2['Latitude'], data2['Longitude'], data2['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_data2)  
    
map_data2

_Neighborhoods are more clustered around the bigger city_

*Markdown added to annotate*